In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## LSI
LSI : represent each document in documentsxvocab (cell value is term frequency) matrix and perform SVD, pick top few dimensions for dimensionality reduction. The each document has a reduced dimensionality vector for clustering and classification. 

The 20 newsgroups dataset comprises around 18000 newsgroups posts on 20 topics split in two subsets: one for training (or development) and the other one for testing (or for performance evaluation). The split between the train and test set is based upon a messages posted before and after a specific date.



In [4]:
from sklearn.datasets import fetch_20newsgroups

In [26]:
dataset = fetch_20newsgroups(shuffle=True, random_state=1, remove=('headers', 'footers', 'quotes'))
documents=dataset.data

In [27]:
len(dataset.data)

11314

In [28]:
news_df = pd.DataFrame({'document':documents})


In [32]:
news_df.loc[0].document

"Well i'm not sure about the story nad it did seem biased. What\nI disagree with is your statement that the U.S. Media is out to\nruin Israels reputation. That is rediculous. The U.S. media is\nthe most pro-israeli media in the world. Having lived in Europe\nI realize that incidences such as the one described in the\nletter have occured. The U.S. media as a whole seem to try to\nignore them. The U.S. is subsidizing Israels existance and the\nEuropeans are not (at least not to the same degree). So I think\nthat might be a reason they report more clearly on the\natrocities.\n\tWhat is a shame is that in Austria, daily reports of\nthe inhuman acts commited by Israeli soldiers and the blessing\nreceived from the Government makes some of the Holocaust guilt\ngo away. After all, look how the Jews are treating other races\nwhen they got power. It is unfortunate.\n"

In [33]:
# removing everything except alphabets`
news_df['clean_doc'] = news_df['document'].str.replace("[^a-zA-Z#]", " ")

# removing short words
news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))

# make all text lowercase
news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: x.lower())

In [34]:
news_df.loc[0].clean_doc

'well sure about story seem biased what disagree with your statement that media ruin israels reputation that rediculous media most israeli media world having lived europe realize that incidences such described letter have occured media whole seem ignore them subsidizing israels existance europeans least same degree think that might reason they report more clearly atrocities what shame that austria daily reports inhuman acts commited israeli soldiers blessing received from government makes some holocaust guilt away after look jews treating other races when they power unfortunate'

### NLTK is needed

In [37]:
pip install nltk

     |████████████████████████████████| 1.4 MB 203 kB/s eta 0:00:01
     |████████████████████████████████| 672 kB 11.6 MB/s eta 0:00:01
  Created wheel for nltk: filename=nltk-3.5-py3-none-any.whl size=1434677 sha256=83be330f68f44300b085e2b30e03d3c38aa3a752d2216e43b5241b23e7ed2aaf
  Stored in directory: /home/sampath/.cache/pip/wheels/ff/d5/7b/f1fb4e1e1603b2f01c2424dd60fbcc50c12ef918bafc44b155
Successfully built nltk
Note: you may need to restart the kernel to use updated packages.


In [41]:
import nltk
from nltk.corpus import stopwords
#nltk.download()
stop_words = stopwords.words('english')

# tokenization
tokenized_doc = news_df['clean_doc'].apply(lambda x: x.split())

# remove stop-words
# result is list of lists
tokenized_doc = tokenized_doc.apply(lambda x: [item for item in x if item not in stop_words])

# de-tokenization
detokenized_doc = []
for i in range(len(news_df)):
    # make document back again
    t = ' '.join(tokenized_doc[i])
    detokenized_doc.append(t)

In [48]:
news_df['clean_doc'] = detokenized_doc

In [53]:
news_df['clean_doc'][0]

'well sure story seem biased disagree statement media ruin israels reputation rediculous media israeli media world lived europe realize incidences described letter occured media whole seem ignore subsidizing israels existance europeans least degree think might reason report clearly atrocities shame austria daily reports inhuman acts commited israeli soldiers blessing received government makes holocaust guilt away look jews treating races power unfortunate'

In [52]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(stop_words='english', 
max_features= 1000, # keep top 1000 terms 
max_df = 0.5, 
smooth_idf=True)

X = vectorizer.fit_transform(news_df['clean_doc'])

X.shape # check shape of the document-term matrix

(11314, 1000)

#### Rows are documents and columns are term (pruned to top 1000)

In [69]:
from sklearn.decomposition import TruncatedSVD

#Apply SVD to documentsxterms matrix and take top 20 singular vectors. 
# TruncatedSVD handles sparse matrix and since tf-idf is sparse its appropriate
# SVD represent documents and terms in vectors 
svd_model = TruncatedSVD(n_components=100, algorithm='randomized', n_iter=100, random_state=122)

svd_model.fit(X)

# top 20 singular vectors, but the dimension of each vector is same as feature dimension. These are just top 20 basis
# these top 20 vectors are topics. 
len(svd_model.components_)

100

In [70]:
svd_model.components_.shape

(100, 1000)

In [71]:
#100 singular vectors which are orthogonal to each other 
len(svd_model.components_[0])

1000

In [72]:
terms = vectorizer.get_feature_names()

In [75]:
len(terms)

1000

In [76]:
for i, comp in enumerate(svd_model.components_):
    terms_comp = zip(terms, comp)
    #find the one with more magnitude in the component vector
    sorted_terms = sorted(terms_comp, key= lambda x:x[1], reverse=True)[:7]
    print("Topic "+str(i)+": ")
    for t in sorted_terms:
        print(t[0])
        print(" ")

Topic 0: 
like
 
know
 
people
 
think
 
good
 
time
 
thanks
 
Topic 1: 
thanks
 
windows
 
card
 
drive
 
mail
 
file
 
advance
 
Topic 2: 
game
 
team
 
year
 
games
 
season
 
players
 
good
 
Topic 3: 
drive
 
scsi
 
disk
 
hard
 
card
 
drives
 
problem
 
Topic 4: 
windows
 
file
 
window
 
files
 
program
 
using
 
problem
 
Topic 5: 
government
 
chip
 
mail
 
space
 
information
 
encryption
 
data
 
Topic 6: 
like
 
bike
 
know
 
chip
 
sounds
 
looks
 
look
 
Topic 7: 
card
 
sale
 
video
 
offer
 
monitor
 
price
 
jesus
 
Topic 8: 
know
 
card
 
chip
 
video
 
government
 
people
 
clipper
 
Topic 9: 
good
 
know
 
time
 
bike
 
jesus
 
problem
 
work
 
Topic 10: 
think
 
chip
 
good
 
thanks
 
clipper
 
need
 
encryption
 
Topic 11: 
thanks
 
right
 
problem
 
good
 
bike
 
time
 
window
 
Topic 12: 
good
 
people
 
windows
 
know
 
file
 
sale
 
files
 
Topic 13: 
space
 
think
 
know
 
nasa
 
problem
 
year
 
israel
 
Topic 14: 
space
 
good
 
card
 
people
 
time
 
nas

In [82]:
sum(svd_model.explained_variance_ratio_*100)

28.11721002751661

In [84]:
pip install umap

  Created wheel for umap: filename=umap-0.1.1-py3-none-any.whl size=3566 sha256=b636bc25bd848a90a68485a10bc0283b39b85c0636b08b3c06ddb2ef9442aaa7
  Stored in directory: /home/sampath/.cache/pip/wheels/d4/13/91/2e752dc8dab5df027854bd33d2b65e1dc5cdc107fd1133990f
Successfully built umap
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import umap

X_topics = svd_model.fit_transform(X)
embedding = umap.UMAP(n_neighbors=150, min_dist=0.5, random_state=12).fit_transform(X_topics)

plt.figure(figsize=(7,5))
plt.scatter(embedding[:, 0], embedding[:, 1], 
c = dataset.target,
s = 10, # size
edgecolor='none'
)
plt.show()